<a href="https://colab.research.google.com/github/rgmartin/greece_tourism_project/blob/main/greece_tourism_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Greece tourism detination analysis



## 1. Web scrapping excel files

In [40]:
! pip install plotly --upgrade
! pip install dash

     |████████████████████████████████| 7.3 MB 5.1 MB/s 
     |████████████████████████████████| 357 kB 20.4 MB/s 
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=3ee850d0683eedc2c6632326d1ca2c8faaa449c6ce780cceef025e41ab29ee4d
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=26f0a9206fb832ac2501a5a6a4806509955bb3ce46a3b039bcd1cec15543eb86
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=3b7bbf95f71a433f5c257692aebda1bda9b6e09a48864d14baa581ece51c375c
  Stored in directory: /root/.cache/pip/wheels/85/5d/4e/7c276b57992951dbe770bf5caad6448d0539c510663aefd2e2
Successfully built dash-core-components dash-html-com

In [41]:
#####Step 1: start by importing all of the necessary packages#####
import requests #requesting URLs
import urllib.request #requesting URLs
import os
import time #setting the speed at which the requests run
import re #regexp string manipulation
import pandas as pd #for simplifying data operations (e.g. creating dataframe objects)
import matplotlib.pyplot as plt #for plotting the scraped data
from bs4 import BeautifulSoup #for web-scraping operations
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_html_components as html
import dash_core_components as dcc
!pip install dash-bootstrap-components
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`



     |████████████████████████████████| 209 kB 5.2 MB/s 


## Origin country and destination regions for incoming tourism in Greece

In [10]:
#global function for plotting incoming tourism to different regions in Greece
def incoming_tourism (key_figure, year = 2020):

  # auxiliary function to find the links of the excel files containing the regional data
  def web_scrapper(url = 'https://insete.gr/perifereies/'):      
      response = requests.get(url) #Connect to the URL using the "requests" package  
      soup = BeautifulSoup(response.text, 'html.parser')
      return  [link.get('href') for link in soup.find_all('a') if ('xlsx' in link.get('href')) ]  

  

  #auxiliary function to extract the region name from the url
  def extract_region_name(url):
    file_w_extension = os.path.basename(url)
    file = os.path.splitext(file_w_extension)[0]  
    return ''.join(x for x in file if x.isalpha())

  # auxiliary function for reading data from a particular region
  def read_region(region_url, key_figure = 0, year = 2020):
    data = requests.get(region_url).content
    xl = pd.ExcelFile(data)
    parameters = {'sheet_name': 2, 'header':[1],'index_col':[0,1]}
    # localise those rows corresponding to the first multindex
    df = xl.parse(**parameters)
    region_name = extract_region_name(region_url)
    # find the indices that correspond to the data for the corresponding year 
    try:
      year_index = [s  for s in df.index.get_level_values(0) if str(year) in s][0]  
      first_idx = df.index.get_level_values(level=0).get_loc(year_index)
      first_idx = first_idx + 2 # correction for the known table format
      list_of_final_idx=df.index.get_level_values(level=0).get_loc(
          'Πηγή: Έρευνα Συνόρων της ΤτΕ, Επεξεργασία INSETE Intelligence'
      )
      final_idx = [i for i, x in enumerate(list_of_final_idx) if (x and (i>first_idx))][0]
      final_idx = final_idx -2  
      df = df.iloc[first_idx:final_idx,key_figure].droplevel(level=0)
      df.name = region_name
      return(df) 
    except:
      return(None)

  # auxiliary function to construct the nodes of the shankey diagram
  def nodes_df(df):
    regions = list(df.columns)  
    countries = list(df.index)  
    nodes = pd.DataFrame(data = regions + countries, columns = ['Label'])  
    nodes.index.name = 'ID'
    cm = plt.get_cmap('tab20')
    n = len(regions  + countries)
    colors = [matplotlib.colors.rgb2hex(rgba) for rgba in cm(range(n))]
    nodes['Color'] = colors
    return nodes  

  # auxiliary function to construct the links of the shankey diagram
  def links_df(nodes, input_df):  
    links = input_df.stack().reset_index()
    links = links.rename(columns = {
        links.columns[0]: 'Origin',
        links.columns[1]: 'Destination',
        links.columns[2]: 'Value',
    })
    links['Origin_id'] = links.apply(lambda row: nodes.index[nodes.Label ==row['Origin']].to_list()[0],axis =1)
    links['Destination_id'] = links.apply(lambda row: nodes.index[nodes.Label ==row['Destination']].to_list()[0],axis =1)  
    links['Color'] = links.apply(lambda row: nodes.Color[nodes.Label ==row['Destination']].values[0],axis =1)        
    return links
  

  excel_links = web_scrapper('https://insete.gr/perifereies/')
  list_of_df = [read_region(url,key_figure,year) for url in excel_links]
  list_of_df = [df for df in list_of_df if not(df is None)]
  inbound_tourism_per_country_df = pd.concat(list_of_df, axis = 1)
  inbound_tourism_per_country_df = inbound_tourism_per_country_df.rename(index = dict(zip(
        
        ['Λοιπές', 'Γερμανία', 'Ην. Βασίλειο', 'Γαλλία', 'Βουλγαρία', 'Ιταλία',
        'Κύπρος', 'Βόρεια Μακεδονία', 'Τουρκία', 'Ολλανδία', 'Πολωνία',
        'Αλβανία', 'Ρουμανία', 'ΗΠΑ ', 'Ελβετία', 'Σερβία', 'Βέλγιο', 'Αυστρία',
        'ΗΠΑ']
        
      , ['Other Countries', 'Germany', 'Un. Kingdom ','France ','Bulgaria ',
        'Italy', 'Cyprus', 'Northern Macedonia', 'Turkey', 'Netherlands',
        'Poland', 'Albania', 'Romania', 'USA', 'Switzerland', 'Serbia',
        'Belgium', 'Austria', 'USA'])) ) 
  
  nodes = nodes_df(inbound_tourism_per_country_df)
  links = links_df(nodes, inbound_tourism_per_country_df)
  fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = nodes['Label'],
      color = list(nodes['Color']),
      hovertemplate='%{value} (thousands of tourists.<extra></extra>)'

    ),
  link = dict(
    source = links['Origin_id'], # indices correspond to labels, eg A1, A2, A1, B1, ...
    target = links['Destination_id'],
    value = links['Value'], 
    color = [f'rgba{tuple(matplotlib.colors.to_rgba(c)[:3] + (0.2,) )}' for c in links['Color']]

  ))])
  fig.update_layout( font_size=12, width=800,height=600, title = f'Inbound tourism for different Greek regions - year {year} (thousands of tourists)')
  #fig.show()
  return fig



In [11]:
incoming_tourism(0,2016)

## Change of inbound tourism shares between Greek regions for the years 2010-2020

In [37]:
def share_of_inbound_tourism(years = (2010, 2020)):
  # auxiliary function to find the links of the excel files containing the regional data
  def web_scrapper(url = 'https://insete.gr/perifereies/'):      
    response = requests.get(url) #Connect to the URL using the "requests" package  
    soup = BeautifulSoup(response.text, 'html.parser')
    return  [link.get('href') for link in soup.find_all('a') if ('xlsx' in link.get('href')) ]  

  def extract_region_name(url):
    file_w_extension = os.path.basename(url)
    file = os.path.splitext(file_w_extension)[0]  
    return ''.join(x for x in file if x.isalpha())

  def read_inbound_tourism(url):
    data = requests.get(url).content
    xl = pd.ExcelFile(data)
    parameters = {'sheet_name': 6, 'header':[3],'index_col':[0,1]}
    df = xl.parse(**parameters)
    series = df.loc['Σύνολο'].iloc[0]
    series.name = extract_region_name(url)
    return series

  excel_links = web_scrapper('https://insete.gr/perifereies/')
  inbound_tourism_df = pd.concat([read_inbound_tourism(url) for url in excel_links], axis = 1)
  inbound_tourism_df.index.name = 'Year'
  inbound_tourism_df.sort_values(axis=1, by=2020,ascending=False,inplace=True)
  inbound_tourism_shares = inbound_tourism_df.div(inbound_tourism_df.sum(axis=1), axis=0)*100
  # visualisation based on https://towardsdatascience.com/create-effective-data-visualizations-of-proportions-94b69ad34410
  # ideas of visualisation https://www.politico.eu/article/europe-tourism-boom-time-overtourism-top-destinations/
  inbound_tourism_shares = inbound_tourism_shares.loc[years[0]:years[1]]
  fig = px.bar(inbound_tourism_shares,x = inbound_tourism_shares.index, y =inbound_tourism_shares.columns,
               labels=dict(value="Inbound tourism share (%)"))
  fig.update_layout( font_size=12, width=800,height=600, title = f'Share of inbound tourism accross Greek regions - years {years[0]}- {years[1]}',
                    legend_title = 'Region')  
  #fig.show()
  return fig

In [38]:
share_of_inbound_tourism((2015,2020))

## Dashboard

In [42]:
#DASHBOARD APPLICATION SET UP
app = dash.Dash(__name__)
#app = dash.Dash(external_stylesheets=[dbc.themes.DARKLY])

#CREATE LAYOUT OF APPLICATION
app.layout = html.Div(children=[
    
    #TITLE
    html.H1('GREECE TOURISM ANALYSIS 2010-2020', 
            style={'textAlign': 'center', 'font-size': 25, 'margin-top': 20, 'margin-bottom': 20}),
                                
    html.Br(),
    
    #YEAR RANGE SLIDER
    html.P('Select the range of years to display:', style={'textAlign': 'center'}),
    dcc.RangeSlider(id='range_slider', min=2010, max=2020, step=1, value=[2010, 2020], 
                    marks={2010: '2010', 2011: '2011', 2012: '2012', 2013: '2013', 2014: '2014',
                          2015: '2015', 2016: '2016', 2017: '2017', 2018: '2018', 2019: '2019', 2020: '2020'}),
    
    html.Br(),
    
    #GRAPH 1
    html.Div([html.H2('Graph #1: Inbound tourism per Greek region', 
                      style={'textAlign': 'center', 'font-size': 18, 'margin-top': 20, 'margin-bottom': 20})]),

    html.Br(),
    
    html.Div(id='slider-output-container'),
    
    html.Div(dcc.Graph(id='graph', responsive = True), 
        style={"width": "70%", "display": "inline-block", "margin-left": "15%", "margin-right": "15%"}),
    
    html.Br(),
    
    
])

#CALLBACK TO RETURN FIG2
@app.callback(dash.dependencies.Output('graph', 'figure'), [dash.dependencies.Input('range_slider', 'value')])
def get_graph(value):        
    fig = share_of_inbound_tourism((value[0],value[1]))
    return fig

#RUN THE APP
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
